In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
from scipy.stats import linregress
from pprint import pprint

#
import censusdata

In [31]:
data_file = os.path.join("resources", "Decade Demographics.csv")
demog_df = pd.read_csv(data_file)
#demog_df.head(10)

In [59]:
#Format data frame
demog_df = demog_df.dropna()
demog_df = demog_df.rename(columns = {"United States!!Total!!Estimate": "United States Total",
                                    "United States!!Foreign born; Entered 2010 or later!!Estimate": "Entered 2010 or later",
                                   "United States!!Foreign born; Entered 2000 to 2009!!Estimate": "Entered 2000 to 2009",
                                   "United States!!Foreign born; Entered before 2000!!Estimate": "Entered before 2000"})
demog_df = demog_df.reset_index(drop = True)

demog_df.head(25)

,Label (Grouping),United States Total,Entered 2010 or later,Entered 2000 to 2009,Entered before 2000
0,Foreign-born population,"44,932,901","11,142,149","11,035,400","22,755,352"
1,Naturalized citizen,51.6%,13.7%,47.6%,72.1%
2,Not a citizen,48.4%,86.3%,52.4%,27.9%
3,Foreign-born population excluding popu...,"44,932,799","11,142,149","11,035,400","22,755,250"
4,Europe,10.4%,8.0%,7.8%,12.8%
5,Asia,31.4%,38.8%,29.3%,28.8%
6,Africa,5.5%,9.1%,6.7%,3.2%
7,Oceania,0.7%,0.9%,0.7%,0.5%
8,Latin America,50.3%,41.5%,54.3%,52.6%
9,Northern America,1.8%,1.7%,1.2%,2.1%


In [64]:
total_population = demog_df.iloc[1,:]
immigrant_age = demog_df.iloc[12:20, :]
median_age = demog_df.iloc[21, :]

#Turn (X) to 0 (columns still add to 100)
immigrant_age = immigrant_age.replace("(X)", "00")

#Remove Percentage Signs, divide by 100
for i in range (1, 5):
    immigrant_age.iloc[:, i] = list(map(lambda x: x[:-1], immigrant_age.iloc[:, i].values))
    immigrant_age.iloc[:, i] = list(map(lambda x: float(x), immigrant_age.iloc[:, i].values))
    immigrant_age.iloc[:, i] = list(map(lambda x: x / 100, immigrant_age.iloc[:, i].values))
    

#immigrant_age["United States Total"] = list(map(lambda x: x[:-1], immigrant_age["United States Total"].values))


immigrant_age.head(20)

,Label (Grouping),United States Total,Entered 2010 or later,Entered 2000 to 2009,Entered before 2000
12,Under 5 years,0.007,0.027,0.000,0.000
13,5 to 17 years,0.050,0.154,0.048,0.000
14,18 to 24 years,0.063,0.134,0.097,0.012
15,25 to 44 years,0.365,0.485,0.529,0.227
16,45 to 54 years,0.196,0.098,0.183,0.249
17,55 to 64 years,0.152,0.053,0.085,0.233
18,65 to 74 years,0.097,0.032,0.037,0.158
19,75 to 84 years,0.050,0.013,0.017,0.085


In [4]:
data_file2 = os.path.join("resources", "Annual Perm Resident 1820-2019.csv")
ann_perm_res = pd.read_csv(data_file2)
ann_perm_res = ann_perm_res.drop(columns = ann_perm_res.columns[0])
ann_perm_res.tail(20)

,Year,Number of Legal Permanent Residents
180,2000,"841,002"
181,2001,"1,058,902"
182,2002,"1,059,356"
183,2003,"703,542"
184,2004,"957,883"
185,2005,"1,122,257"
186,2006,"1,266,129"
187,2007,"1,052,415"
188,2008,"1,107,126"
189,2009,"1,130,818"


In [5]:
data_file3 = os.path.join("resources", "Form type Status.csv")
form_type = pd.read_csv(data_file3)
form_type = form_type.iloc[1: , 1: ]

form_type.head()

,Category and Form Number,Description,Forms,Unnamed: 4,Completed,Unnamed: 6,Unnamed: 7,Processing,Forms.1,Unnamed: 10,Completed.1,Unnamed: 12,Unnamed: 13
1,NaN,NaN,Received,Approved,Denied,Total 15,NaN,Time,Received,Approved,Denied,Total 15,NaN
2,TOTAL - ALL FORMS,NaN,"2,343,364","1,905,587","251,700","2,176,453","8,037,262",NaN,"8,839,722","6,138,783","808,323","7,010,652","8,037,262"
3,Family Based,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,I-129F,Petition for Alien Fiancé(e),"10,965","6,284","2,265","8,549","30,408",8.0,"37,741","24,445","6,639","31,084","30,408"
5,I-130,Petition for Alien Relative,"216,241","153,184","16,445","169,629","1,519,906",10.8,"757,206","673,956","81,169","755,125","1,519,906"


In [6]:
data_file4 = os.path.join("resources", "Lawful Permanent Status.csv")
perm_status = pd.read_csv(data_file4)
perm_status = perm_status.iloc[1: , 1: ]

perm_status.head()

,Type and class of admission,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
1,Total,"1,062,040","1,031,631","990,553","1,016,518","1,051,031","1,183,505","1,127,167","1,096,611","1,031,765","707,362"
2,Immediate relatives of U.S. citizens,"453,158","478,780","439,460","416,456","465,068","566,706","516,508","478,961","505,765","321,148"
3,Spouses,"258,320","273,429","248,332","238,852","265,367","304,358","292,909","268,149","304,334","195,718"
4,Children1,"80,311","81,121","71,382","61,217","66,740","88,494","74,989","66,794","61,303","37,379"
5,Parents,"114,527","124,230","119,746","116,387","132,961","173,854","148,610","144,018","140,128","88,051"


In [7]:
data_file5 = os.path.join("resources", "Refugee Admissions.csv")
refugee_admit = pd.read_csv(data_file5)

refugee_admit.head(20)

,Year,Annual Ceiling,Number of Admitted Refugees
0,1975,-,"146,158"
1,1976,-,"27,206"
2,1977,-,"19,946"
3,1978,-,"36,507"
4,1979,-,"111,363"
5,1980,"231,700","207,116"
6,1981,"217,000","159,252"
7,1982,"140,000","98,096"
8,1983,"90,000","61,218"
9,1984,"72,000","70,393"


In [20]:
src = "sf1"
year = 2019 #years work only <= 2019 for acs



#sample = censusdata.search(src, year,'concept', 'immigration')
sample = censusdata.search("acs5", year, 'concept', 'transportation')